<a href="https://colab.research.google.com/github/s1250103/Public_mori-lab/blob/eras/eras/tune/programs/prototypeModelMaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2021/01/08：
全結合層のみのモデル

#共通設定

In [2]:
%tensorflow_version 2.x

##import

In [12]:
# file dealing
import os
import shutil
from google.colab import drive
# data dealing
import numpy as np
from pandas import DataFrame
import matplotlib.pyplot as plt
# process deasing
import gc
from time import sleep

# machine learning (back)
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras import layers, models, initializers, callbacks
from tensorflow.keras.layers import SimpleRNN

# machine learning
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.utils.np_utils import to_categorical


seed = 20201218
np.random.seed(seed)
tf.random.set_seed(seed)

##スクラッチ

In [16]:
#@title デフォルトのタイトル テキスト
del model
keras.backend.clear_session()
gc.collect()

NameError: ignored

## データフォーマットを規定

In [4]:
class video_format:
  name = "video_format"
  # サンプリングされたCMデータの仕様
  playtime = "15秒"
  displaysize = "(any, any, RGB)"
  videoformat = "any"
  # モデルが扱うCMデータ(上のようなデータは、下のように変換される)
  HEIGHT = 45
  WIDTH = 80
  FRAME_SIZE = 30
  COLORinfo = 3 # "RGB"
  FPS = "2 (FRAME_SIZE / playtime)" # 定義ではなく上から導かれた値

format1 = video_format()

#モデル作成

In [46]:
# モデル作成
model = models.Sequential()
# 入力層
model.add(
     layers.Reshape(
        (format1.FRAME_SIZE,
         format1.HEIGHT,
         format1.WIDTH,
         format1.COLORinfo),
        input_shape=(format1.FRAME_SIZE * format1.HEIGHT * format1.WIDTH * format1.COLORinfo,),
        name='ENTRANCE' )
)
# 中間層
## dropout_0
# model.add(layers.Dropout(0.2))

## 畳み込み0
model.add(
    layers.Conv3D(
        filters=32,
        kernel_size=(2, 3, 3),
        strides=(1, 1, 1),
        padding='same',
        activation='relu',
        name='conv0'))
## pool0
model.add(
    layers.MaxPooling3D(pool_size=(2, 3, 3), name='pool0'))

## 畳み込み1
model.add(
    layers.Conv3D(
        filters=64,
        kernel_size=(2, 3, 3),
        strides=(1, 1, 1),
        padding='same',
        activation='relu',
        name='conv1'))
## pool1
model.add(
    layers.MaxPooling3D(pool_size=(2, 3, 3), name='pool1'))

# ## 畳み込み2
# model.add(
#     layers.Conv3D(
#         filters=128,
#         kernel_size=(2, 3, 3),
#         strides=(1, 1, 1),
#         padding='same',
#         activation='relu',
#         name='conv2'))
# ## pool2
# model.add(
#     layers.MaxPooling3D(pool_size=(2, 3, 3), name='pool2'))

## dropout_1
# model.add(layers.Dropout(0.2))

## 全結合0
model.add(
    layers.Flatten(name='flatten'),
)
model.add(
    layers.Dense(1024,
      activation='relu',
      name='dence0' ),
)
# 出力層
model.add(
    layers.Dense(4, activation='softmax', name='WATERSUPPLY')
)

# 最適化
adam = keras.optimizers.Adam(
    lr=1e-7, beta_1=0.9, beta_2=0.999,
    epsilon=None, decay=0.0, amsgrad=False)
# 誤差関数
mse = tf.keras.losses.MeanSquaredError()
cce = tf.keras.losses.CategoricalCrossentropy()

model.compile(
    optimizer=adam,
    loss=cce,
    metrics=['acc', cce])

In [ ]:
plot_model(model, 
           show_shapes=True,
           show_layer_names=False,
           to_file='model.png')

In [75]:
print(type(model))


<class 'tensorflow.python.keras.engine.sequential.Sequential'>


# 表示

In [ ]:
model.summary()
plot_model(model ,show_shapes=True,)

In [45]:
sleep(10)
del model
keras.backend.clear_session()
gc.collect()

4374